In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

from enum import Enum

In [2]:
#存檔位置
ModelMode = "ResNet_ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[4] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[4] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[4]

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)
    #篩選出ESC10作為訓練資料
    df_10 = df[df['esc10']==True]
    #取出類別，去除重複並排序
    classes = df_10['category'].unique()
    class_dict = {i:x for x,i in enumerate(classes)}
    #重新標label
    df_10['target'] = df_10['category'].map(class_dict)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df_10[df_10.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df_10))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#殘差網路模塊
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        shortcut = self.shortcut(input)
        input = nn.ReLU()(self.bn1(self.conv1(input)))
        input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = input + shortcut
        return nn.ReLU()(input)
    
#ResNet_18+注意力機制 模型
class ResNet_18(nn.Module):
    def __init__(self, nums_class=10):
        super(ResNet_18, self).__init__()
        
        self.layer_0 = nn.Sequential(
            # width_of_output = (width_of_input - filter_size + 2*padding)/stride + 1
            nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = (7,7), stride = (2,2), padding = 3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        #殘差層
        #-----------------------------------------------------
        self.layer_1 = nn.Sequential(
            ResBlock(64, 128, downsample=True),
            ResBlock(128, 128, downsample=False)
        )
        
        self.layer_2 = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.Linear(64, nums_class)
        )
        
    def forward(self, x):
        x = self.layer_0(x)
        x = self.layer_1(x)
        x = self.layer_2(x)
        return x



In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 60
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = 0.0032
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_3828\4284335736.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


C:\Users\test\AppData\Local\Temp\ipykernel_3828\4284335736.py:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)
C:\Users\test\AppData\Local\Temp\ipykernel_3828\4284335736.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_3828\4284335736.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


Progress: 100% - Completed
Data Loading Time: 28.28 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = ResNet_18(nums_class = 10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #5s
            dataX = dataX5
            dataY = dataY5
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
        else:
            #1s
            dataX = dataX1
            dataY = dataY1
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 1.5021523237228394 Train_acc: 32.812500%
Train Epoch: 2/60 Train_Loss: 1.1948918104171753 Train_acc: 51.250000%
Train Epoch: 3/60 Train_Loss: 1.0044903755187988 Train_acc: 56.875000%
Train Epoch: 4/60 Train_Loss: 0.9185012578964233 Train_acc: 68.437500%
Train Epoch: 5/60 Train_Loss: 0.7738775014877319 Train_acc: 76.250000%
Train Epoch: 6/60 Train_Loss: 0.8823515176773071 Train_acc: 72.187500%
Train Epoch: 7/60 Train_Loss: 0.6712637543678284 Train_acc: 75.625000%
Train Epoch: 8/60 Train_Loss: 0.5399235486984253 Train_acc: 78.750000%
Train Epoch: 9/60 Train_Loss: 0.6312201023101807 Train_acc: 80.937500%
Train Epoch: 10/60 Train_Loss: 0.6346476078033447 Train_acc: 71.250000%
Train Epoch: 11/60 Train_Loss: 0.

Train Epoch: 46/60 Train_Loss: 0.06650276482105255 Train_acc: 97.604172%
Train Epoch: 47/60 Train_Loss: 0.055713947862386703 Train_acc: 97.812508%
Train Epoch: 48/60 Train_Loss: 0.08481007069349289 Train_acc: 97.604172%
Train Epoch: 49/60 Train_Loss: 0.0655733048915863 Train_acc: 97.708336%
Train Epoch: 50/60 Train_Loss: 0.08274472504854202 Train_acc: 97.916672%
Train Epoch: 51/60 Train_Loss: 0.05129249393939972 Train_acc: 98.125008%
Train Epoch: 52/60 Train_Loss: 0.0626467689871788 Train_acc: 98.020836%
Train Epoch: 53/60 Train_Loss: 0.02634594775736332 Train_acc: 98.333336%
Train Epoch: 54/60 Train_Loss: 0.016268290579319 Train_acc: 97.291672%
Train Epoch: 55/60 Train_Loss: 0.0890302062034607 Train_acc: 97.500008%
Train Epoch: 56/60 Train_Loss: 0.061864543706178665 Train_acc: 98.333336%
Train Epoch: 57/60 Train_Loss: 0.05318843573331833 Train_acc: 97.916672%
Train Epoch: 58/60 Train_Loss: 0.08909972757101059 Train_acc: 97.916672%
Train Epoch: 59/60 Train_Loss: 0.15618938207626343 Tra

Train Epoch: 29/60 Train_Loss: 0.22600619494915009 Train_acc: 91.250000%
Train Epoch: 30/60 Train_Loss: 0.3040618598461151 Train_acc: 91.562500%
Train Epoch: 31/60 Train_Loss: 0.19782155752182007 Train_acc: 90.625000%
Train Epoch: 32/60 Train_Loss: 0.2837083041667938 Train_acc: 91.875000%
Train Epoch: 33/60 Train_Loss: 0.2879849672317505 Train_acc: 92.812500%
Train Epoch: 34/60 Train_Loss: 0.22965161502361298 Train_acc: 91.250000%
Train Epoch: 35/60 Train_Loss: 0.2054818570613861 Train_acc: 93.750000%
Train Epoch: 36/60 Train_Loss: 0.16957645118236542 Train_acc: 94.062500%
Train Epoch: 37/60 Train_Loss: 0.12554548680782318 Train_acc: 93.437500%
Train Epoch: 38/60 Train_Loss: 0.2688294053077698 Train_acc: 92.187500%
Train Epoch: 39/60 Train_Loss: 0.18437695503234863 Train_acc: 93.437500%
Train Epoch: 40/60 Train_Loss: 0.19140738248825073 Train_acc: 92.500000%
Train Epoch: 41/60 Train_Loss: 0.20263232290744781 Train_acc: 94.687500%
Train Epoch: 42/60 Train_Loss: 0.1488434374332428 Train_

Train Epoch: 12/60 Train_Loss: 0.4986211359500885 Train_acc: 86.125000%
Train Epoch: 13/60 Train_Loss: 0.7780613899230957 Train_acc: 85.500000%
Train Epoch: 14/60 Train_Loss: 0.2092178612947464 Train_acc: 86.875000%
Train Epoch: 15/60 Train_Loss: 0.2548905313014984 Train_acc: 87.250000%
Train Epoch: 16/60 Train_Loss: 0.29326462745666504 Train_acc: 87.937500%
Train Epoch: 17/60 Train_Loss: 0.17465737462043762 Train_acc: 90.937500%
Train Epoch: 18/60 Train_Loss: 0.18540741503238678 Train_acc: 89.750000%
Train Epoch: 19/60 Train_Loss: 0.27034127712249756 Train_acc: 90.625000%
Train Epoch: 20/60 Train_Loss: 0.13245436549186707 Train_acc: 91.437500%
Train Epoch: 21/60 Train_Loss: 0.2311078906059265 Train_acc: 90.437500%
Train Epoch: 22/60 Train_Loss: 0.21757663786411285 Train_acc: 91.000000%
Train Epoch: 23/60 Train_Loss: 0.2295389324426651 Train_acc: 91.250000%
Train Epoch: 24/60 Train_Loss: 0.1873847246170044 Train_acc: 91.437500%
Train Epoch: 25/60 Train_Loss: 0.14669084548950195 Train_a

Test accuracy of the model: 73.750000%
Test accuracy of the model: 80.000000%
Training Time: 8.93 seconds
----------------------------------------------------------------------------------------------------------------
fold:  3 TL:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 1.3071881532669067 Train_acc: 66.250000%
Train Epoch: 2/60 Train_Loss: 0.6478699445724487 Train_acc: 75.729172%
Train Epoch: 3/60 Train_Loss: 0.48505380749702454 Train_acc: 77.500008%
Train Epoch: 4/60 Train_Loss: 0.6278145909309387 Train_acc: 84.375008%
Train Epoch: 5/60 Train_Loss: 0.49127310514450073 Train_acc: 84.687508%
Train Epoch: 6/60 Train_Loss: 0.45695778727531433 Train_acc: 85.000008%
Train Epoch: 7/60 Train_Loss: 0.23869673907756805 Train_acc: 85.833336%
Train Epoch: 8/60 Train_Loss: 0.3490981161594391 Train_acc: 90.000008%
Train Epoch: 9/60 Train_Loss: 0.2774050235748291 Train_acc: 89.583336%
Train Epo

Train Epoch: 44/60 Train_Loss: 0.3131158649921417 Train_acc: 94.125000%
Train Epoch: 45/60 Train_Loss: 0.11395736783742905 Train_acc: 93.500000%
Train Epoch: 46/60 Train_Loss: 0.12052248418331146 Train_acc: 94.187500%
Train Epoch: 47/60 Train_Loss: 0.08482173830270767 Train_acc: 94.312500%
Train Epoch: 48/60 Train_Loss: 0.14421674609184265 Train_acc: 94.562500%
Train Epoch: 49/60 Train_Loss: 0.09613817930221558 Train_acc: 94.375000%
Train Epoch: 50/60 Train_Loss: 0.18898995220661163 Train_acc: 94.562500%
Train Epoch: 51/60 Train_Loss: 0.10778967291116714 Train_acc: 93.750000%
Train Epoch: 52/60 Train_Loss: 0.17531265318393707 Train_acc: 94.375000%
Train Epoch: 53/60 Train_Loss: 0.15085475146770477 Train_acc: 94.437500%
Train Epoch: 54/60 Train_Loss: 0.13415883481502533 Train_acc: 93.812500%
Train Epoch: 55/60 Train_Loss: 0.05102209001779556 Train_acc: 94.187500%
Train Epoch: 56/60 Train_Loss: 0.19327448308467865 Train_acc: 94.250000%
Train Epoch: 57/60 Train_Loss: 0.12104398757219315 T

Train Epoch: 27/60 Train_Loss: 0.12126490473747253 Train_acc: 96.458336%
Train Epoch: 28/60 Train_Loss: 0.10876534134149551 Train_acc: 96.041672%
Train Epoch: 29/60 Train_Loss: 0.22637857496738434 Train_acc: 96.250008%
Train Epoch: 30/60 Train_Loss: 0.08075017482042313 Train_acc: 97.187508%
Train Epoch: 31/60 Train_Loss: 0.12297654896974564 Train_acc: 97.395836%
Train Epoch: 32/60 Train_Loss: 0.03935724124312401 Train_acc: 97.604172%
Train Epoch: 33/60 Train_Loss: 0.024929232895374298 Train_acc: 97.812508%
Train Epoch: 34/60 Train_Loss: 0.08857907354831696 Train_acc: 97.812508%
Train Epoch: 35/60 Train_Loss: 0.10979689657688141 Train_acc: 97.708336%
Train Epoch: 36/60 Train_Loss: 0.08258773386478424 Train_acc: 96.875008%
Train Epoch: 37/60 Train_Loss: 0.03578115254640579 Train_acc: 97.916672%
Train Epoch: 38/60 Train_Loss: 0.07709527760744095 Train_acc: 97.916672%
Train Epoch: 39/60 Train_Loss: 0.05561278760433197 Train_acc: 98.125008%
Train Epoch: 40/60 Train_Loss: 0.05751892551779747

Train Epoch: 11/60 Train_Loss: 0.38119304180145264 Train_acc: 79.687500%
Train Epoch: 12/60 Train_Loss: 0.6882641911506653 Train_acc: 81.562500%
Train Epoch: 13/60 Train_Loss: 0.6951026916503906 Train_acc: 81.875000%
Train Epoch: 14/60 Train_Loss: 0.4992522597312927 Train_acc: 82.812500%
Train Epoch: 15/60 Train_Loss: 0.47408583760261536 Train_acc: 82.500000%
Train Epoch: 16/60 Train_Loss: 0.29873064160346985 Train_acc: 82.187500%
Train Epoch: 17/60 Train_Loss: 0.36823493242263794 Train_acc: 84.687500%
Train Epoch: 18/60 Train_Loss: 0.4537666440010071 Train_acc: 86.250000%
Train Epoch: 19/60 Train_Loss: 0.3661712110042572 Train_acc: 86.250000%
Train Epoch: 20/60 Train_Loss: 0.39239558577537537 Train_acc: 86.250000%
Train Epoch: 21/60 Train_Loss: 0.26068243384361267 Train_acc: 90.625000%
Train Epoch: 22/60 Train_Loss: 0.26864099502563477 Train_acc: 90.937500%
Train Epoch: 23/60 Train_Loss: 0.3451528549194336 Train_acc: 90.625000%
Train Epoch: 24/60 Train_Loss: 0.2964341342449188 Train_a

Train Epoch: 59/60 Train_Loss: 0.13425733149051666 Train_acc: 97.395836%
Train Epoch: 60/60 Train_Loss: 0.01846010982990265 Train_acc: 97.812508%
Test accuracy of the model: 74.000000%
Test accuracy of the model: 85.833333%
Test accuracy of the model: 90.000000%
Training Time: 16.02 seconds
----------------------------------------------------------------------------------------------------------------
fold:  5 TL:  2
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 0.8089155554771423 Train_acc: 71.375000%
Train Epoch: 2/60 Train_Loss: 0.6536815762519836 Train_acc: 78.750000%
Train Epoch: 3/60 Train_Loss: 0.4903879463672638 Train_acc: 81.937500%
Train Epoch: 4/60 Train_Loss: 0.29019397497177124 Train_acc: 83.312500%
Train Epoch: 5/60 Train_Loss: 0.5140785574913025 Train_acc: 83.687500%
Train Epoch: 6/60 Train_Loss: 0.4368000626564026 Train_acc: 80.562500%
Train Epoch: 7/60 Train_Loss: 0.3469130

In [9]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()